# Creating Training Loop

## VERSIONS
- 00_01: 
    - Diagnosing Exploding Gradients
- 00_00: 
    - Initial Version

## Imports

In [1]:
# from importlib.metadata import version
import pandas as pd
# import seaborn as sn
from pathlib import Path
import os
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torch.nn import Module # For type hinting
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

## Data Preparation

### Custom Dataset

In [2]:
class WeatherDataset(Dataset):
    """Dataset class For the CA Weather Fire Dataset"""
    def __init__(self, csv_file="../Data/CA_Weather_Fire_Dataset_Cleaned.csv"):
        try:
            self.data = pd.read_csv(csv_file)   # Assign a pandas data frame
        except FileNotFoundError:
            raise FileNotFoundError(f"File not found: {csv_file}")

        # Define feature and label columns
        self.feature_columns = self.data.columns.drop("MAX_TEMP")
        self.label_column = "MAX_TEMP"
        

    def __getitem__(self, index):
        features = self.data.loc[index, self.feature_columns].values
        
        label = self.data.loc[index, self.label_column] # Extract the label for the given index
        return (
            torch.tensor(features, dtype=torch.float),
            torch.tensor(label, dtype=torch.float)
        )

    def __len__(self):
        return len(self.data)

### Data Pipeline

In [15]:
def data_pipeline(root_data_dir: str= "../Data", data_file_path: str="CA_Weather_Fire_Dataset_Cleaned.csv", data_splits_dir: str="DataSplits", batch_size: int=64, num_workers=0, pin_memory: bool=False, drop_last: bool=True) -> tuple[Dataset, Dataset, Dataset, DataLoader, DataLoader, DataLoader, StandardScaler]:
    """This function prepares the train, test, and validation datasets.
    Args:
        root_data_dir (str): The root of the Data Directory
        data_file_path (str): The name of the original dataset (with .csv file extension).
        data_splits_dir (str): Path to the train, test, and validation datasets.
        batch_size (int): The dataloader's batch_size.
        num_workers (int): The dataloader's number of workers.
        pin_memory (bool): The dataloader's pin memory option.
        drop_last (bool): The dataloader's drop_last option.

    Returns: 
        train_dataset (Dataset): Dataset Class for the training dataset.
        test_dataset (Dataset): Dataset Class for the test dataset.
        validation_dataset (Dataset): Dataset Class for the validation dataset.
        train_dataloader (DataLoader): The train dataloader.
        test_dataloader (DataLoader): The test dataloader.
        validation_dataloader (DataLoader): The validation dataloader.
        scaler (StandardScaler): The scaler used to scale the features of the model input.
        """
    
    if not root_data_dir or not data_file_path or not data_splits_dir:  # Check for empty strings at the beginning
        raise ValueError("File and directory paths cannot be empty strings.")
    
    WEATHER_DATA_DIR = Path(root_data_dir)                  # Set the Data Root Directory

    WEATHER_DATA_CLEAN_PATH = WEATHER_DATA_DIR / data_file_path # Set the path to the complete dataset

    if WEATHER_DATA_CLEAN_PATH.exists():
        print(f"CSV file detected, reading from {WEATHER_DATA_CLEAN_PATH}")
        df = pd.read_csv(WEATHER_DATA_CLEAN_PATH)
    else:
        print(f"Downloading csv file from HuggingFace")
        try:
            df = pd.read_csv("hf://datasets/MaxPrestige/CA_Weather_Fire_Dataset_Cleaned/Data/CA_Weather_Fire_Dataset_Cleaned.csv")  # Download and read the data into a pandas dataframe
            os.makedirs(WEATHER_DATA_DIR, exist_ok=True)        # Create the Data Root Directory
            df.to_csv(WEATHER_DATA_CLEAN_PATH, index=False)     # Save the file, omitting saving the index
        except Exception as e:
            raise RuntimeError(f"An unexpected error occurred during data download or saving: {e}")
    
    DATA_SPLITS_DIR = WEATHER_DATA_DIR / data_splits_dir
    TRAIN_DATA_PATH = DATA_SPLITS_DIR / "train.csv"
    TEST_DATA_PATH = DATA_SPLITS_DIR / "test.csv"
    VALIDATION_DATA_PATH = DATA_SPLITS_DIR / "val.csv"
    SCALER_PATH = DATA_SPLITS_DIR / "scaler.joblib"

    features = ['DAY_OF_YEAR', 'PRECIPITATION', 'LAGGED_PRECIPITATION', 'AVG_WIND_SPEED', 'MIN_TEMP']
    target = 'MAX_TEMP'

    if os.path.exists(TRAIN_DATA_PATH) and os.path.exists(TEST_DATA_PATH) and os.path.exists(VALIDATION_DATA_PATH) :
        print(f"Train, Test, and Validation csv datasets detected in '{DATA_SPLITS_DIR}', skipping generation")
        scaler = joblib.load(SCALER_PATH)
    else:
        print(f"Datasets not found in '{DATA_SPLITS_DIR}' or incomplete. Generating datasets...")
        os.makedirs(DATA_SPLITS_DIR, exist_ok=True)     # Create the Data Splits Parent Directory

        X = df[features]
        y = df[target]

        # split your data before scaling, shuffling the data
        X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
        X_test, X_validation, y_test, y_validation = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

        # Initialize the StandardScaler
        scaler = StandardScaler()

        # Fit the scaler on the training data ONLY. Need to use the scaler on all inputs that the model receives.
        # This means the mean and standard deviation are calculated from the training set.
        scaler.fit(X_train)

        # Transform the training, validation, and test data using the fitted scaler
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        X_validation_scaled = scaler.transform(X_validation)

        # Save the fitted scaler object
        joblib.dump(scaler, SCALER_PATH)

        X_train_df = pd.DataFrame(X_train_scaled, columns=features)
        X_test_df = pd.DataFrame(X_test_scaled, columns=features)
        X_validation_df = pd.DataFrame(X_validation_scaled, columns=features)

        # Concatenate the features and labels back into a single DataFrame for each set
        train_data_frame = pd.concat([X_train_df, y_train.reset_index(drop=True)], axis=1)
        test_data_frame = pd.concat([X_test_df, y_test.reset_index(drop=True)], axis=1)
        validation_data_frame = pd.concat([X_validation_df, y_validation.reset_index(drop=True)], axis=1)

        # Saving the split data to csv files
        train_data_frame.to_csv(TRAIN_DATA_PATH, index=False)
        test_data_frame.to_csv(TEST_DATA_PATH, index=False)
        validation_data_frame.to_csv(VALIDATION_DATA_PATH, index=False)

    print(f"Initializing DataLoaders and Returning")
    # Initialize the Different Datasets
    train_dataset = WeatherDataset(TRAIN_DATA_PATH)
    test_dataset = WeatherDataset(TEST_DATA_PATH)
    validation_dataset = WeatherDataset(VALIDATION_DATA_PATH)
    # Initialize the Different DataLoaders using the Datasets
    print(f"Creating DataLoaders with batch_size ({batch_size}), num_workers ({num_workers}), pin_memory ({pin_memory}). Training dataset drop_last: ({drop_last})")
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, drop_last=drop_last, shuffle=True)
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, drop_last=drop_last)
    validation_dataloader = DataLoader(dataset=validation_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, drop_last=drop_last)

    print(f"Training DataLoader has ({len(train_dataloader)}) batches, Test DataLoader has ({len(test_dataloader)}) batches, Validation DataLoader has ({len(validation_dataloader)}) batches")
    
    return (train_dataset, test_dataset, validation_dataset, train_dataloader, test_dataloader, validation_dataloader, scaler)
        

## Agent Architecture

### Layer Block

In [23]:
class LayerBlock(torch.nn.Module):
    """Class for the individual layer blocks."""
    def __init__(self, intermediate_dim=32):
        super().__init__()
        self.Layer1 = torch.nn.Linear(in_features=intermediate_dim, out_features=intermediate_dim)
        self.ReLu = torch.nn.ReLU()
        self.Layer_Norm1 = torch.nn.LayerNorm(normalized_shape=intermediate_dim)

    def forward(self, x):
        x = self.Layer1(x)
        x = self.ReLu(x)
        x = self.Layer_Norm1(x)
        return x

### Weather Agent

In [24]:
class WeatherAgent(torch.nn.Module):
    """Class for Agent Structure using multiple Layer Blocks."""
    def __init__(self, cfg):
        super().__init__()
        self.L1 = torch.nn.Linear(in_features=cfg["in_dim"], out_features=cfg["intermediate_dim"])
        
        self.Layers = torch.nn.Sequential(
            *[LayerBlock(cfg["intermediate_dim"]) for _ in range(cfg["num_blocks"])]
        )
        self.out = torch.nn.Linear(in_features=cfg["intermediate_dim"], out_features=cfg["out_dim"])

    def forward(self, x):
        x = self.L1(x)
        x = self.Layers(x)
        x = self.out(x)
        return x

## Main

### Log Iteration Functions

In [6]:
def log_iteration(batch_idx: int, total_batches: int, loss_value: float):
    """Logs the loss of the current batch."""
    print(f"Epoch batch [{batch_idx}/{total_batches}] | Loss: {loss_value:.7f}")

In [7]:
def log_epoch_iteration(epoch: int, avg_epoch_loss: float):
    """Log Current Metrics accumulated in the current epoch iteration.
    Args:
        epoch (int): the current iteration
        avg_epoch_loss (float): The average loss of the current epoch
    Returns:
        N/A
        """
    if avg_epoch_loss:
        print(f"=====================  [EPOCH ({epoch}) LOGGING]  =====================")
        print("| AVERAGES of THIS EPOCH:")
        print(f"| ACCUMULATED LOSS: {avg_epoch_loss:.7f}")
        print(f"===========================================================")
    
    else:
        print("No Data collected for this epoch to log")

### Evaluate Model Function

In [8]:
def evaluate_model(model: Module, dataloader: DataLoader, current_epoch: int = None, max_epochs: int=None, device: str = 'cpu') -> float:
    """
    Evaluates the model on a given dataset and returns the average loss.
    Args:
        model (Module): The Model.
        dataloader (DataLoader): The dataloader to calculate average loss with.
        current_epoch (int): The current epoch [optional].
        max_epochs (int): The maximum number of epochs [optional].
        device (str): The device that the calculations will take place on.
    Returns:
        avg_loss (float): The calculated average loss.
    """
    model.eval()
    total_loss = 0.0
    loss_fn = torch.nn.MSELoss(reduction='sum') # Use reduction='sum' instead of 'mean' for total loss

    if len(dataloader.dataset) == 0:
        print("Warning: Evaluation dataset is empty. Skipping evaluation.")
        return float('nan')
    
    with torch.no_grad():
        for batch_inputs, batch_labels in dataloader:
            batch_inputs, batch_labels = batch_inputs.to(device), batch_labels.unsqueeze(dim=-1).to(device)
            outputs = model(batch_inputs)
            loss = loss_fn(outputs, batch_labels)
            total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader.dataset)     # Calculate the average loss on the dataset

    if current_epoch and max_epochs:   # If the function was called in the training loop
        print(f"===================  [Epoch ({current_epoch}/{max_epochs})]  ===================")
        print(f"Entire Validation Dataset Average Loss: {avg_loss:.4f}")
        print(f"====================================================")

    else:   # If the function was called outside of the training loop
        print(f"===============================================")
        print(f"Entire Dataset Average Loss: {avg_loss:.4f} ")
        print(f"=====================================================")
            
    return avg_loss

### Train Model Function

In [33]:
def train_model(model_config: dict, train_dataloader: DataLoader, validation_dataloader: DataLoader, model: WeatherAgent = None, epochs=32, learning_rate=0.0003, max_grad_norm=0.5, log_iterations=10, eval_iterations=10, device="cpu") -> WeatherAgent:
    """The Model Training function.

    Args:
        model_config (dict): The base configurations for building the policies.
        train_dataloader (DataLoader): The dataloader for the training loop.
        validation_dataloader (DataLoader): The dataloader for the validation loop.
        model (WeatherAgent): The model to be trained.
        epochs (int): The number of times the outer loop is performed.
        learning_rate (float): The hyperparameter that affects how much the model's parameters learn on each update iteration.
        max_grad_norm (float): Used to promote numerical stability and prevent exploding gradients.
        log_iterations (int): Used to log information about the state of the Agent.
        eval_iterations (int): Used to run an evaluation of the Agent.
        device (str): The device that the model will be trained on.

    Returns: 
        agent (Module): The Trained Model in evaluation mode.
    """
    print(f"Training Model on {device} with {epochs} main epochs, {learning_rate} learning rate, max_grad_norm={max_grad_norm}.")
    print(f"Logging every {log_iterations} epoch iterations, evaluating every {eval_iterations} epoch iterations.")

    agent = (model if model is not None else WeatherAgent(model_config)).to(device) # Create agent if nothing was passed, otherwise, create the agent. Send agent to device.

    optimizer = torch.optim.Adam(params=agent.parameters(), lr=learning_rate)   # Define the model optimization algorithm
    loss_fn = torch.nn.MSELoss(reduction='mean')       # Define the Loss function

    history = {'train_loss': [], 'val_loss': []}

    train_dataloader_length= len(train_dataloader)
    agent.train()   # Set agent to training mode
    for epoch in tqdm(range(epochs), desc=f">>>>>>>>>>>>>>>>>>>>>\nMain Epoch (Outer Loop)", leave=True):

        epoch_loss_total = 0.0
        for batch_idx, (inputs, labels) in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs} - Training", leave=False)):           # Get a mini-batch of training examples from the dataloader
            # optimizer.zero_grad(set_to_none=True)       # Clear the gradients built up; Setting to None to improve performance
            optimizer.zero_grad()       # Clear the gradients built up; Setting to None to improve performance

            inputs, labels = inputs.to(device), labels.unsqueeze(dim=-1).to(device)   # Move the inputs and labels to the device

            agent_outputs = agent(inputs)       # Pass the inputs to the model and get the outputs.

            # print(f"Model Outputs:")
            # print(f"{agent_outputs.squeeze()}")

            # print(f"labels:")
            # print(f"{labels.squeeze()}")

            loss = loss_fn(agent_outputs, labels)      # Calculate the mini-batch loss
            # print(f"Current Loss {loss.item()}")
            epoch_loss_total += loss.item()
            
            loss.backward()         # Calculate the loss with respect to the model parameters
            # torch.nn.utils.clip_grad_norm_(parameters=agent.parameters(), max_norm=max_grad_norm)   # Prevent the gradients from affecting the model parameters too much and reduce the risk of exploding gradients

            # --- ADD GRADIENT MONITORING HERE ---
            # total_norm = 0
            # for p in agent.parameters():
            #     if p.grad is not None:
            #         param_norm = p.grad.data.norm(2)
            #         total_norm += param_norm.item() ** 2
            # total_norm = total_norm ** 0.5
            # print(f"Total Gradient Norm: {total_norm}")
            # ------------------------------------
            optimizer.step()      # Update the model's parameters using the learning rate

            # LOGGING LOSS OF CURRENT ITERATION
            if (batch_idx + 1) % log_iterations == 0:
                log_iteration(batch_idx=(batch_idx + 1), total_batches=train_dataloader_length, loss_value=loss.item())

        # CALCULATE AND STORE THE AVERAGE EPOCH LOSS
        epoch_avg_loss = epoch_loss_total / train_dataloader_length
        history["train_loss"].append(epoch_avg_loss)

        # LOG THE AVERAGE LOSS OF THE EPOCH
        log_epoch_iteration(epoch=epoch, avg_epoch_loss=epoch_avg_loss)

        # EVALUATE THE MODEL
        if (epoch + 1) % eval_iterations == 0:
            val_loss = evaluate_model(model=agent, dataloader=validation_dataloader, current_epoch=(epoch + 1), max_epochs=epochs, device=device)
            history["val_loss"].append(val_loss)
            agent.train()   # Set agent to training mode

        
    return agent.eval(), history

### Testing

In [60]:
model_config={
    "in_dim": 12,
    "intermediate_dim": 32,
    "out_dim": 1,
    "num_blocks": 2
}

In [48]:
agent = WeatherAgent(model_config)

In [49]:
agent

WeatherAgent(
  (L1): Linear(in_features=12, out_features=32, bias=True)
  (Layers): Sequential(
    (0): LayerBlock(
      (Layer1): Linear(in_features=32, out_features=32, bias=True)
      (ReLu): ReLU()
      (Layer_Norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    )
    (1): LayerBlock(
      (Layer1): Linear(in_features=32, out_features=32, bias=True)
      (ReLu): ReLU()
      (Layer_Norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    )
  )
  (out): Linear(in_features=32, out_features=1, bias=True)
)

In [50]:
input=torch.rand(size=(2,12))

In [51]:
output = agent(input)

In [52]:
output

tensor([[-0.8667],
        [-0.7486]], grad_fn=<AddmmBackward0>)

## Main

In [37]:
try:
    (train_dataset, test_dataset, validation_dataset, train_dataloader, test_dataloader, validation_dataloader, scaler) = data_pipeline(batch_size=256)
except ValueError as e:
    print(f"Caught an error: {e}")

CSV file detected, reading from ..\Data\CA_Weather_Fire_Dataset_Cleaned.csv
Train, Test, and Validation csv datasets detected in '..\Data\DataSplits', skipping generation
Initializing DataLoaders and Returning
Creating DataLoaders with batch_size (256), num_workers (0), pin_memory (False). Training dataset drop_last: (True)
Training DataLoader has (46) batches, Test DataLoader has (5) batches, Validation DataLoader has (5) batches


In [31]:
model_config={
    "in_dim": 5,
    "intermediate_dim": 32,
    "out_dim": 1,
    "num_blocks": 5
}

In [35]:
trained_model, hist = train_model(model_config=model_config, train_dataloader=train_dataloader, validation_dataloader=validation_dataloader, epochs=128, learning_rate=0.001, max_grad_norm=1.0, log_iterations=64, eval_iterations=16)

Training Model on cpu with 128 main epochs, 0.001 learning rate, max_grad_norm=1.0.
Logging every 64 epoch iterations, evaluating every 16 epoch iterations.


>>>>>>>>>>>>>>>>>>>>>
Main Epoch (Outer Loop):   0%|          | 0/128 [00:00<?, ?it/s]

Epoch batch [64/93] | Loss: 4355.3339844


>>>>>>>>>>>>>>>>>>>>>
Main Epoch (Outer Loop):   1%|          | 1/128 [00:01<03:45,  1.78s/it]

=====================  [EPOCH (0) LOGGING]  =====================
| AVERAGES of THIS EPOCH:
| ACCUMULATED LOSS: 4474.3207010


...

>>>>>>>>>>>>>>>>>>>>>
Main Epoch (Outer Loop):   5%|▌         | 7/128 [00:12<03:36,  1.79s/it]

=====================  [EPOCH (6) LOGGING]  =====================
| AVERAGES of THIS EPOCH:
| ACCUMULATED LOSS: 379.7492905


Epoch batch [64/93] | Loss: 156.9589081


>>>>>>>>>>>>>>>>>>>>>
Main Epoch (Outer Loop):   5%|▌         | 7/128 [00:14<04:02,  2.01s/it]


KeyboardInterrupt: 

In [18]:
trained_model

WeatherAgent(
  (L1): Linear(in_features=5, out_features=32, bias=True)
  (Layers): Sequential(
    (0): LayerBlock(
      (Layer1): Linear(in_features=32, out_features=32, bias=True)
      (ReLu): ReLU()
    )
  )
  (out): Linear(in_features=32, out_features=1, bias=True)
)

In [20]:
total_params=sum(p.numel() for p in trained_model.parameters() if p.requires_grad)

In [21]:
print(total_params)

1281


In [38]:
trained_model, hist = train_model(model_config=model_config, train_dataloader=train_dataloader, validation_dataloader=validation_dataloader, epochs=128, learning_rate=0.0001, max_grad_norm=1.0, log_iterations=128, eval_iterations=16, device='cuda')

Training Model on cuda with 128 main epochs, 0.0001 learning rate, max_grad_norm=1.0.
Logging every 128 epoch iterations, evaluating every 16 epoch iterations.


>>>>>>>>>>>>>>>>>>>>>
Main Epoch (Outer Loop):   1%|          | 1/128 [00:01<02:36,  1.23s/it]

=====================  [EPOCH (0) LOGGING]  =====================
| AVERAGES of THIS EPOCH:
| ACCUMULATED LOSS: 4907.8337827


>>>>>>>>>>>>>>>>>>>>>
Main Epoch (Outer Loop):   2%|▏         | 2/128 [00:02<02:34,  1.22s/it]

=====================  [EPOCH (1) LOGGING]  =====================
| AVERAGES of THIS EPOCH:
| ACCUMULATED LOSS: 4781.1647418


...

=====================  [EPOCH (125) LOGGING]  =====================
| AVERAGES of THIS EPOCH:
| ACCUMULATED LOSS: 586.6289686


>>>>>>>>>>>>>>>>>>>>>
Main Epoch (Outer Loop):  99%|█████████▉| 127/128 [02:44<00:01,  1.25s/it]

=====================  [EPOCH (126) LOGGING]  =====================
| AVERAGES of THIS EPOCH:
| ACCUMULATED LOSS: 565.5932471


>>>>>>>>>>>>>>>>>>>>>
Main Epoch (Outer Loop): 100%|██████████| 128/128 [02:45<00:00,  1.30s/it]

=====================  [EPOCH (127) LOGGING]  =====================
| AVERAGES of THIS EPOCH:
| ACCUMULATED LOSS: 546.5657733
===================  [Epoch (128/128)]  ===================
Entire Validation Dataset Average Loss: 460.5843


In [27]:
inputs = torch.tensor([0.06062194469261507,-0.1858927462585465,-0.3487493382395144,0.2855731423742254,0.9586116733014429], dtype=torch.float).unsqueeze(0)
# target: 74.0

In [28]:
model_output = trained_model(inputs)

In [29]:
model_output

tensor([[74.5019]], grad_fn=<AddmmBackward0>)

In [30]:
inputs2 = torch.tensor([1.4353250258453352,-0.1858927462585465,-0.3487493382395144,-1.2838959675367174,-1.2536259299793566], dtype=torch.float).unsqueeze(0)
# target: ,72.0
model_output2 = trained_model(inputs2)
print(model_output2)

tensor([[68.7885]], grad_fn=<AddmmBackward0>)
